# Own model
This Jupyter notebook is divided into three main parts. The first part under the Markdown [“Google Colab”](#google-colab) contains the necessary preparation and instructions to train the models in Google Colab. The second part under the Markdown [“Local”](#local) contains the necessary preparation and instructions to train the models locally. The third and last part under the Markdown ["Model"](#models) contains all models which are functional with the respective preparation in Google Colab as well as locally.

# Google Colab
The code below is intended for execution in Google Colab. To execute the code in Google Colab you have to comment out the above import for Google Colab and compress the already restructured folder into a zip file and upload “Dataset_Original.zip” to your Google Drive.

You can then extract the folder with the ZIP extractor.

<img src="ImageLib/GoogleDriveZip.jpg" alt="GoogleDrive image" style="width:700px;"/>


In [ ]:
#Imports to work in Google Colab
from google.colab import drive 
drive.mount('/content/drive') 

import tensorflow as tf
import matplotlib.pyplot as plt

In the below Code first defines the paths for the training and test data directories. The image size is set to 224x224 pixels and the image shape takes into account three color channels (RGB). The batch size for processing is 32 images.
When loading the datasets, the training dataset is split into 75% training data and 25% validation data, while the test dataset contains all images from the test directory. The data sets are randomly mixed, whereby a seed value ensures reproducibility.
The class names are extracted from the training dataset and output. A normalization layer is used to scale the image pixels to a range of 0-1, and this normalization is applied to all datasets. Caching and prefetching of the datasets is used to increase efficiency.
Finally, data augmentation is defined that includes random augmentations such as horizontal flipping, rotation and zooming to make the model more robust and avoid overfitting.

In [ ]:
# Directories for training and test data
train_dir = '/content/drive/MyDrive/ML2/Dataset_Original/train'
test_dir = '/content/drive/MyDrive/ML2/Dataset_Original/test'

# Image sizes and batch size
IMAGE_SIZE = (224, 224) 
IMAGE_SHAPE = IMAGE_SIZE + (3,) 
BATCH_SIZE = 32


# Load the images from the directories
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=999,
    validation_split=0.25,
    subset="training"
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=999,
    validation_split=0.25,
    subset="validation"
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=999
)

class_names = train_dataset.class_names
print("Klassennamen:", class_names)


# Add standardization layer
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Normalize the datasets
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# Prefetching und Caching for efficiency
train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Definiere die Datenaugmentierungsschichten
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

# Local

This code can be executed locally. It uses the data augmentation script [“dataPreparation.py”](dataPreparation.py). A more detailed description of the script can be found below under the markdown [Data preparation](#data-preparation).

Below you will find the necessary imports for the local execution of the code

In [ ]:
#Imports to work local
import tensorflow as tf
import matplotlib.pyplot as plt
from dataPreparation import load_and_prepare_datasets, get_data_augmentation

### Data preparation
Please check that the path to your data set matches.

The Code first defines the paths for the training and test data directories. The image size is set to 224x224 pixels and the image shape takes into account three color channels (RGB). The batch size for processing is 32 images.

In the function "load_and_prepare_datasets" the dataset is loaded, when loading the datasets, the training dataset is split into 75% training data and 25% validation data, while the test dataset contains all images from the test directory. The data sets are randomly mixed, whereby a seed value ensures reproducibility. Afterwards, the class names are extracted from the training dataset. Finally, a normalization layer is used to scale the image pixels to a range of 0-1, and this normalization is applied to all datasets. Caching and prefetching of the datasets is used to increase efficiency.


The function "get_data_augmentation" includes random augmentations such as horizontal flipping, rotation and zooming to make the model more robust and avoid overfitting.

In [1]:
# Directories for training and test data
train_dir = 'Dataset_Original/train'
test_dir = 'Dataset_Original/test'


# Image sizes and batch size
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
BATCH_SIZE = 32

# Load and prepare the dataset
train_dataset, validation_dataset, test_dataset, class_names = load_and_prepare_datasets(train_dir, test_dir, IMAGE_SIZE, BATCH_SIZE)

# Show class names
print("Klassennamen:", class_names)

# Get the data augmentation layers
data_augmentation = get_data_augmentation()

Found 1178 files belonging to 3 classes.
Using 884 files for training.
Found 1178 files belonging to 3 classes.
Using 294 files for validation.
Found 393 files belonging to 3 classes.
Klassennamen: ['gls', 'nlb', 'nls']


# Models

In this Section you will find 4 Models.

[Model 1](#model-1)

[Model 2](#model-2)

[Model 3](#model-3)

[Model 4](#model-4)


### Model 1

In [ ]:
# Define the CNN model 1
model_1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=IMAGE_SHAPE),
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3)
])
model_1.summary()

# Compile the model
model_1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Train the model
history = model_1.fit(
    train_dataset,
    epochs=9,
    validation_data=validation_dataset
)

# Evaluate
test_loss, test_acc = model_1.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_acc}')


# Visualize the training progress
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.title('Training and Validation Accuracy')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.legend(loc='lower right')

plt.subplot(2, 1, 2)
plt.title('Training and Validation Loss')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('epoch')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.legend(loc='upper right')

plt.suptitle('Training and Validation Metrics Model 1', fontsize=16)
plt.show()

### Model 2

In [ ]:
# Define the CNN model 2
model_2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=IMAGE_SHAPE),
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation='softmax')  # Für Mehrklassenklassifikation
])
model_2.summary()

# Compile the model
model_2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Train the model
history = model_2.fit(
    train_dataset,
    epochs=20,
    validation_data=validation_dataset
)

# Evaluate
test_loss, test_acc = model_2.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_acc}')


# Visualize the training progress
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.title('Training and Validation Accuracy')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.legend(loc='lower right')

plt.subplot(2, 1, 2)
plt.title('Training and Validation Loss')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('epoch')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.legend(loc='upper right')

plt.suptitle('Training and Validation Metrics Model 2', fontsize=16)
plt.show()

### Model 3

In [ ]:
# Define the CNN model 3
model_3 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=IMAGE_SHAPE),
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])
model_3.summary()

# Compile the model
model_3.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Train the model
history = model_3.fit(
    train_dataset,
    epochs=20,
    validation_data=validation_dataset
)

# Evaluate
test_loss, test_acc = model_3.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_acc}')


# Visualize the training progress
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.title('Training and Validation Accuracy')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.legend(loc='lower right')

plt.subplot(2, 1, 2)
plt.title('Training and Validation Loss')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('epoch')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.legend(loc='upper right')

plt.suptitle('Training and Validation Metrics Model 3', fontsize=16)
plt.show()

### Model 4

In [ ]:
# Define the CNN model
model_4 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=IMAGE_SHAPE),
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])
model_4.summary()

# Learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile the model
model_4.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(filepath='modelLib/modelTraining/best_model.keras', save_best_only=True, monitor='val_loss'),
]

# Train the model
history = model_4.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20,
    callbacks=callbacks
)

# Evaluate
test_loss, test_acc = model_4.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_acc}')


# Visualize the training progress
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.title('Training and Validation Accuracy')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.legend(loc='lower right')

plt.subplot(2, 1, 2)
plt.title('Training and Validation Loss')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('epoch')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.legend(loc='upper right')

plt.suptitle('Training and Validation Metrics Model 4', fontsize=16)
plt.show()

### Safe
Use the following code to save the model and class names if you want to use the application with frontend and backend.

In [11]:
#Safe model
model_4.save('modelLib/model_4.keras')

#Safe class names
#with open('modelLib/class_names.txt', 'w') as f:
#    for class_name in class_names:
#        f.write("%s\n" % class_name)